# Recurrent Neural Network
- RNN are very effective for NLP and other sequence tasks because they have "memory".  They can read input $x^{<t>}$ one at a time, and remember some information through the hidden layer activations. It has several variants including LSTMs, GRUs and Bidirectional RNNs.

## Notation
- Superscript[l] denotes an object associated with the $l^{th}$ layer
- Superscript (i) denotes an object associated with the $i^{th}$ example
- Superscript <t> denotes an object at the $t^{th}$ time step.
- $T_x$ is the size of the input sequence and $T_y$ is the size of the output sequence. 
- $x^{(i)<t>}$ is the t_th element in the input sequence of i-th training example. Similarly, $y^{(i)<t>}$ means the t-th element in the output sequence of i-th training example. 

## Representing words

i. We need a vocabulary list that contains all the words in our target sets.
   - Each world will have a unique index that it can be represented with.
   - Vocabulary size in modern applications are from 30,000 to 50,000. 100,000 is not uncommon. Some of the bigger companies use even a million. 
   - To build a vocabulary list, you can read all the texts you have and get m words with the most occurrence. 
    
ii. Create a one-hot coding sequence for each world in your dataset given the vocabulary you have created.
 
 - While converting, if we meet a word that's not in your dictionary, we can add a token in the vocabulary with name ```<UNK> ``` which stands for unkown text and use its index for your one-hot vector. 
 
## Recurrent Neural Network
 - Why not use a  standard network for sequence tasks?
     - Inputs, outputs can be different lengths in different examples
         - This can be solved for normal NNs by padding withe maximum lengths but it's not a good solution. 
         
    - CNN Doesn't share features learned across different positions of text/sequence. For example, the neural network has learned that maybe the word "harry" appearing in position one gives a sign that that's part of a person's name. Then it will be nice if the NN automatically figures out that "Harry" appearing in some other position also means taht that might be a person's name. 


- A RNN scans from left to right through a data, the parameters used in each time steps are shared. 
- When make the prediction for $y^{<t>}$, it gets the information not only from $x^{<t>}$ but also the information/inputs earlier in the sequence $x^{<t-1>},x^{<t-2>}, etc$.

### Forward Propagation
- Forward Propagation
    - $a^{<0>} = 0$
 
    - $a^{<t>} = g(W_{aa}a^{<t-1>} + W_{ax}x^{<t>} + b_a)$, usually tanh  activation, sometimes relu
   
    - $y^{<t>} = g(W_{ya}a^{<t>}+b_y)$, either sigmoid/ softmax activation function      

- Simplied RNN notation
    - $a^{<t>} = g(W_a[a^{<t-1>}, x^{<t>}] + b_a$
    - $y^{<t>} = g(W_y a^{<t>} + b_y)$
    - W<sub>a</sub> is W<sub>aa</sub> and W<sub>ax</sub> stacked horizontally
    - [$a^{<t-1>}$,$x^{<t>}$] is $a^{<t-1>}$ and $x^{t}$ stacked vertically
    - $W_a$ shape:(#ofHiddenNeurons, #ofhiddenNeurons+n<sub>x</sub>)
    - [$a^{<t-1>}$,$x^{<t>}$] shape: (#ofHiddenNeurons+n<sub>x</sub>,1)
    
### Backward Propogation
- We will use the cross-entropy loss function
    - $L^{<t>}(\hat{y}^{<t>},y^{<t>}) = - y^{<t>}log\hat{y}^{<t>} - (1-y^{<t>})log(1-\hat{y}^{<t>})$
- Loss for the entire sequence:
    - $L(\hat{y},y) = \sum^{T_y}_{t=1}L^{<t>}(\hat{y}^{<t>},y^{<t>})$

# Language model and sequence generation
- RNNs do very well in language model problems.
- The job of language model is to a gvie a probability of any given sequnce of words. 
## How to build language models with RNNs?
- The first thing is to get a training set: a large corpus of language text. 
- Then tokenize the traning set by getting the vocabulary and then one-hot each word.
    - Put an end of sentence token ```<EOS>``` in the vocabulary and include it with each converted sentences. Also, use token ```<UNK>``` for unknown words. 
    
- The loss function is defined by cross-entropy loss:
     - $L(\hat{y}^{<t>},y^{<t>}) = - \sum_{i} y_i^{<t>}log\hat{y}_{i}^{<t>}$
     - $L = \sum L^{<t>}(\hat{y}^{<t>},y^{<t>})$
     - i if for all elements in the dictionary,each output$\hat{y}^{<t>}$ is a softmax with size equal to the dictionalry.
     - t for all timesteps
## Sampling novel sequences

- After a sequence model is trained, to check what the model has learned you can apply it to generate a sample novle sequence, to do so:
    - First pass $a^{<0>} = 0$ and $x^{<1>} = zero$
    - Then choose a prediction randomly from distribution obtained by $\hat{y}^{<1>}$.
        - in numpy this can be implemented using ```numpy.random.choice(..)```
        - This is hte line where you get a random beginning of the sentence each time you sample run a novel sequence. 
- We pass the last predicted word with the calculated $a^{<1>}$
- We keep doing for a fixed length or until we get ```<EOS>``` token

- So far, we have build a word-level language model. It's also possible to implement a character level language model.
- Pros and cons of character level model:
    - Pros:
        - There will be no ```<UNK>``` token
    - Cons:
        - The main advantage is that you end up with much longer sequences
        - Character-level language models are not as good as world level language models at capturing long range dependencies between how the earlier parts of the sentence also affect the later part of the sentence. 
        - More computationally expensive and harder to train
        
## Vanishing gradients with RNNs
 - Language can have very long dependicies. Basic RNNs is not very good at capturing very long term dependencies. 
- As we have discussed in Deep neural networks, deeper networks are getting into the vanishing gradient problem. That also happens with RNNs with a long sequence size
 - In backprop, it can be quite difficult, because of the same vanishing gradients problem,  for the outputs of the errors associated with the later time steps to affect the computations that are earlier.
 - In practice, what this means is, it might be difficult to get a neural network to realize that it needs to memorize they just see a singular noun or a plural noun, so that later on in the sequence that can generate either was or were. 
 - Vanishing gradients problems tends to be the bigger problem with RNNs than the exploding gradients problem. 
 - Exploding gradients can be easily seen when your weights values become $NaN$. 
 - Solutions for the Exploding gradient problem:
     - Truncated backpropagation
         - Not to update all the weights in the way back
         - Not optimal. You won't update all the weights
     - Gradient Clipping
          - if your gradient is more than some threshold, rescale some of your gradient vector so that it is not too big
- Solution for the Vanishing gradient problem:
    - Weight initialization
        - like He initialization
    - Echo state networks
    - Use LSTM/GRU network
    
### Gated Recurrent Unit(GRU)

- GRU is an RNN type that can help solve the vanishing gradient prolbem and can remember the long-term dependicies. 
- The basic RNN unit:
    - $a^{<t>} = g(W_a[a^{<t-1>},x^{<t>}]+ b_a)$
    - $y^{<t>} = g'(a^{<t>})$
- The GRU unit  has a new variable $c$, which stands for memory cell. And what the memory cell do is it will provide a bit of memory to remember somehting. 

#### Simplified GRU unit
- At each timestep:
    - $c^{<t-1>} = a^{<t-1>}$
    - An candidate to replace $c^{<t>}$ is $\tilde{c}^{<t>}=tanh(w_c[c^{<t-1>},x^{<t>}]+b_c)$
    - A update gate to deterime  whether we replace $c^{t}$ with $\tilde{c}^{t}$ or not  is  $\Gamma _u =sigmoid(w_u[c^{<t-1>},x^{t}]+b_u) $, $\Gamma_u$ is always between 0 to 1. 
    - $ c^{<t>} = \Gamma_u * \tilde{c}^{t} +(1-\Gamma_u) * c^{<t-1>}$
    - $a^{<t>} = c^{<t>}$
    - $\hat{y}^{<t>}=softamx(a^{t})$
- Because the update gate $\Gamma_u$ is usually a small number like 0.00001, which makes $c^{<t>} = c^{<t-1>}$ in many timesteps,  GRUs doesn't suffer the vanishing gradient problem. Which make the RNNs to learn long range dependicies.  

#### Full GRU unit:
- The full GRU unit contains a new gate that is used to calculate the candidate $\tilde{c}_t$.The gate tells you how relevant is $c^{<t-1>}$ to $c^{<t>}$
- Equations:
    - $c^{<t-1>} = a^{<t-1>}$
    -  $\tilde{c}^{<t>}=tanh(w_c[\Gamma_r *c^{<t-1>},x^{<t>}]+b_c)$
    - update gate $\Gamma _u =sigmoid(w_u[c^{<t-1>},x^{t}]+b_u) $, $\Gamma_u$ is always between 0 to 1. 
    - Relevant gate $\Gamma_r = sigmoid(w_r[c^{<t-1>},x^{<t>}] + b_r)$
    - $c^{<t>} = \Gamma_u * \tilde{c}^{t} +(1-\Gamma_u) * c^{<t-1>}$    
- Good reference  http://colah.github.io/posts/2015-08-Understanding-LSTMs/      

### Long Short Term Memory(LSTM)
- Another type of unit that allows you to learn the long term dependicies very well is LSTM, and LSTM is even more powerful the GRU. 
- In LSTM, $C^{<T>}!= a^{<t>}$
- Here are the equations for an LSTM unit:
    - $\tilde{c}^{<t>} = tanh(W_c[a^{<t-1>},x^{<t>}] + b_c)$
    - $\Gamma_u = \sigma(W_u[a^{<t-1>},x^{<t>}]+b_u)$  update gate
    - $\Gamma_f = \sigma(W_f[a^{<t-1>},x^{<t>}]+b_f)$  forget gate
    - $ \Gamma_o = \sigma(W_o[a^{<t-1>},x^{<t>}]+b_o)$ output gate
    - $c^{<t>} = \Gamma_u * \tilde{c}^{t} + \Gamma_o * c^{<t-1>}$
    - $a^{<t>} = \Gamma_o * tanh c^{<t>}$
- In GRU, we have an update gate, relevant gate, and a candidate cell variables; 
- While in LSTM we have an update gate, forget gate, output gate and a candidate cell variables 
- Some variants on LSTM includes:
    - LSTM wiht peephole connections:
        - The normal LSTM with $c^{<t-1>}$ included wiht every gate
- There isn't a univeral superior between LSTM and GRU. one of the advantages of GRU is that it's simple and can ge used to build much bigger network but the LSTM is more powerful and general. 


## Bidirectional RNN
- BiRNN have a forward activation sequence $a^{<t>}_{forward}$and backward activation sequence $a^{<t>}_{backward}$ 
- The prediction $\hat{y}^{<t>} = g(W_y[a^{<t>}_{forward},a^{<t>}_{backward}] + b_y)$
- The disadvantages of BiRNNs is that you need the entire sequence before processing. 

## Deep RNNs
- Deep RNNs stack some RNN layers to make a deeper netwrok 
- In regular deep nets, there could be 100 or even 200 layers. In deep RNNs stacking 3 layers is already considered deep and expensive to train. 
- In some cases, you might see some standdard deep layers connected after recurrent layers. These standard deep layers are not connected horizontally.

## Natural Language Processing and Word Embeddings
- Word embedding is a way of representing words. It lets your algorithm automatically understand the analogies between words like "king" and "queen"
- Besides one hot vector, another way to represent  word  is using feature vector. This representation is called world embedding. 
- To visualize word emdedding we can use t-SNE algorithm to reduce the features to 2 dimentions which makes it easy to visualize. 

- Cosine similarity - the most commonly used similarity function:
    - $sim (\vec{u}, \vec{v}) = \frac{u^T v}{\vert\vert u\vert \vert _2 \vert\vert v \vert \vert _2} $
    - The top part represents the inner product of $\vec{u}$ and $\vec{v}$. 
- You can also use Euclidean distance $\vert\vert \vec{u} - \vec{v} \vert \vert _2$  as a similarity function(but it rather measures dissimilarity, so you should take it with negative sign)   
### Learning word embeddings
#### Neural language model

- The language modeling problems poses a machine learning problem where you input the context(like the last four words) and predict some target words. And posing this problem allows you to learn good word embeddings.
- We want to build a langauge model to predict the next word. $ I want a glass of orange __$
- We use a neural network to learn the language model
- In the example, we took a window of 6 words that fall behind the word taht we want to predict. There are other choices when we are tryint to learn word embeddings:
    - Suppose we have an example, "I want a glass of orange juice to go along with my cereal"
    - To learn juice, choices of context are:
        - last 4 words
        - 4 words on the left and 4 on the right
        - Last 1 word
        - Nearby 1 word
            -"glass" is near juice
            - This is the idea of skip grams model
- Research found that if you really want to build a language model, it's natural to use the last few words as a context. But if your main goal is really to learn a word embedding,  then you can use 
#### Word2Vec Skip-gram model

- In skip-gram model, given a training sentence example, we are going to come up with a few context to target pairs to create  the supervised learning problem. 

- Rather than having the context be the last four words or the last ten words immediately before the target word, randomly pick a word to be the context word, and randomly pick another word within some window.
- The supervised learning problem will be given the context word, predict the target word. 
- By setting up this supervized learning problem, we will learn good word embeddings. 
- Word2Vec model:
    - Vocabulary size = 10, 000words
    - we want to learn $c$ to get $t$
    - We get $e_c$ by $ E \cdot O_c$
    - a softmax layer is used to get $p(t\vert c)$, which is $\hat{y}$
    - Cross- entropy loss function are used
- Softmax function over 10,000 words is very expensive to compute
- To speed up softmax classfication,  we can use"Hierarchical softmax classsfier" which works as a tree classsfier. 
- In practive, the hierarchical softmax classfier doesn't use a balanced/symmetric tree, but common words are at the top and less common ones are at the bottom.
- How to sample the context? Once the context are chosen, the target word will be within a window from the context word
    - In practive, we don't take the context uniformly random, instead there are some heuristics to balance the common words and the non-common words.
    
#### Negative sampling
- A new supervised learning problem is posed: given a pair of words, predict is this a context -target pair?
- Positive examples are generated by using the skip-gram technique, with a fixed window that goes around. 
- Negative examples are generated by picking a word randomly from the vocabulary. 
- The steps to generate the training samples are:
    - pick a positive context
    - pick k negative contexts from the dictionary,k is recommended to be from 5 to 20 for smaller datasets, from 2 to 5 for larger datasets. 
- Now let's define the supervised learning model to learn a mapping from $x$ to $y$    
     - Let's say that the context word is $c$ and the target word is $t$ and $y$ is the lable
     - $p(y=1 \vert c,t) = \sigma (\theta_t^T e_c)$
     - Instead of having one gaint 10,000 softmax, which is very expesive to compute, we turned it into 10,000 binary classfication problems, each of which is cheap to compute. And on every iteration, we're only going to train $k+1$ of them. $k$ is the number of negatie examples. Thus the compution cost is much lower. 
- How to select negative samples:
    - We can sample according to the empirical frequencies in words corpus. 
    - The best is to sample with this equation
        - $ p(w_i) = \frac{f(w_i)^{3/4}}{\sum_{i=1}^{j=10,000} f(w_i)^{3/4}}$
#### GloVe(global vectors for word representation)

- GloVe is simplest algorithm for learning the word embeddings. 
- A context and target pair is chosen as in Word2Vec
- Calculate $X_{ct}$, which is the # of times t appears in close proximity of c. 
- $X_{ct} = X_{tc}$ if we choose t within a window, they are not euqal if we choose the previous words for examle.
- The model is defined as : 
    - minimize $\sum_{i=1}^{10,000}\sum_{j=1}^{10,000}f(x_{ij})(\theta_i^{T}e_j+b_i+b_j - logX_{ij})^2$
    - f(x)  - weighting term, used for reasons include:
        - let $f_{ij} = 0$ when $X_{ij} =0$, solve the log(0) problem. By convention, $0log0 = 0$
        - Giving too much weight for words with high frequency
        - Giving too little weight for infrequent word
- You can't guarantee that the axis used to represent the features will be humanly explainable. 
        

### Application using word embeddings

#### Sentiment Classfication
- One of the challenge with sentiment classfication is that you might not have a huge labeled training data for it, but using word embedding can help a lot.
- RNN model can be used. 

#### Debiasing word embeddings

- Word embeddings can reflect gender, ethnicity, age, sexual orientation, and other biases of the text used to train the model
- To reduce bias
    - Identify bias direction
    - Neutralize: For every word that is not definitional, project onto non bias direction to reduce the compents in the  bias direction. 
    - Equalize pairs: manually adjust the distance from definitional words to neural words, and make them equal. For example, mom and dad should have the same distance to "computer-programmer"

## Sequence to Sequence model
- Many to many basic models
- Use an RNN/LSTM/GRU to encode the input sentence to an vector, then feed this vector to an decoder RNN, and ouput an new sequence. 
- An similar architecture is ued for image captioning. An AlexNet is used for encoding, and RNN is used for decoding. 


### Beam Search
- Instead of ouput the random sequence, we want the the best output sequence. 
- Beam search is the most widely used algorithm to get the best output sequence. It's a heuristic search algorithm. 
- The algorithm has a parameter $B$ which is the beam width. The algorithm will get B output at a time. 
- The first thing Beam search has to do is try to pick the first word. Greedy search will pick only the one most likely words. Beam search will consider  number of beam width most likely words at one time. 

### Refinement to Beam Search
- Length optimization
    - In beam search, we are trying to optimize 
        $argmax p (y^{<1>},...y^{<T_y>}\vert x)$
    - To do that we mutiply $P(y^{<1>}\vert x) *P(y^{<2>}\vert x, y^{<1>})* .... *P(y^{<T_y>}\vert x, y^{<1>}, ...y^{<T_y -1>})$
    - Each probability is a fraction, most to the time a small fraction, Multiplying small fractions will cause a numerical underflow, meaning that it's too small for the floating part representation to store accurately
    - So in practive, we use summing logs of probalilites, the optimization became:
        $argmax \sum_{y=1}^{T_y}log(p^{<t>}\vert x, y^{<1>}, ... y^{<t-1>})$
    - But the two optimization functions ww have here are perferrring short sequence to max $p$.
    - To Solve this problem, we divide by the number of elments in the sequence:
        $\frac{1}{T_y^\alpha }\sum_{y=1}^{T_y}log(p^{<t>}\vert x, y^{<1>}, ... y^{<t-1>})$
         - $\alpha$ is a hyperparamter to tune.
         - In practive $\alpha = 0.7 $ is a good choice. 
- How to choose best $B$?
    -The larger $B$, the better the results but the more computionally expensive
    - In practice, you might see in production $B =10$
    -$B=100 or 300$ are used sometimes in research
    - Unlike exact search algorithms like BFS or DFS, Beam search runs faster but is not guaranted to find the exact solution. 

### Error analysis in Beam Search
- We can use error analysis ot figure out whether is the beam search or RNN cause bad performance. 
- Let $y^{*}$ be the human generated translation result and $\hat{y}$ be the machine generated translation. 
- If $p(y^{*}\vert x)$ > $p(\hat{y}\vert x)$, beam search is at fault
    - Because it fails to choose the result with a higher probability
    - To fix it, we can try bigger $B$
- If  $p(y^{*}\vert x)$ <= $p(\hat{y}\vert x)$ , RNN is at fault
    
    - To fix it, we can try regulation, more training data, or different architecture
- To do the error analysis, we go over the dev set, find the ones that didn't get the good performance, find which is at higher fraction, beam search of RNN, then decice which to work on next. 

### BLEU(Bilingual Evaluation understudy) score
- One of the challenges of machine translation, is that given a sentence in a language there are one or more possible good translation that are all good. How do we evluate our results?
- One way to measure how good the machine translation output is to look at each of the words in the MT output and see if it appears in the references. This is called a precision of the machine translation output. 
- But this is not a useful measure, so we use Modified precision, in which we look for the refrence with the maximum number of a particular word and clip the maxium appearing of this word to be this number.  
    $\frac{count of number of times the word appears}{the count of number of word in total}$
- For n-gram precision: 
    - $p_n = \frac{\sum_{ngram \in \hat{y}}count_{clip}(ngram)}{\sum_{ngram\in\hat{y}}count(ngram)}$
- The BLEU score:
    - $P_n$- Bleu score on one type of n-gram
    - Combined BLEU score = BP * exp(1/n *sum(p_n))
        - BP is called Brevity Penalty. It turns out that if a machine ouputs a small number of words it will get a better score so we need to fix that: 
        - BP = 1, if MT_ouput_length > reference _output _length
             = exp(1- MT_output_length/reference_output_length)    otherwise
    - BLEU has several open source implemantations.
    
### Attention Model   
- So far we were models with one RNN as encoder and anther RNN as  decoder. There is a technique called attention which makes these models work even better. 
- Since encoder memorize the long sequence into one vector, and the decoer process this vector to generate the translation. The performance of this model get worse if a sentence is long.
- Attention model pay attentio to only part of an input sentence while it's generating a translation. 
- Implementation of Attention model:
    - First we will use an Bidirectional RNN (or GRU or LSTM) to compute features on each word. 
    - Let $a^{<t>}$ include both forward and backward activations at time step $t$
    - context $c$ is computed using the attention weights
        - Sum of the attention weights for each word should be 1: $\sum_{t}\alpha ^{<1,t>} =1$
        - The context $c$ is the weighted average of actionvation from different time steps: $c^{<1>} = \sum_t\alpha^{<1,t>}a^{<t>}$
        - $\alpha^{<t,t'> }$ = amount of attention $\hat{y}^{<t>}$ should pay to $a^{<t'>}$
            - $\alpha^{<t,t'>} = \frac{exp(e^{<t,t'>})}{\sum_{t'=1}^{T_x}exp(e^{<t,t'>})}$
        - A little neural network(usually 1 layer) is used to compute $e^{<t,t'>}$
            - $s^{<t-1>}$ and $\alpha^{<t'>}$ are the input of the small neural network and $e^{<t,t'>}$ is the output. 
            - If you want to decide how much attention to pay to the activation of word at $t'$, it should depend the most on what is your activation from the previous time state$s^{<t-1>}$ and also each of the positions.
            - Use backpropogation and gradient descent to train a very small NN to learn what the functions should be  to get $e^{<t,t'>}$
    - We will use a unidirectional RNN to produce the output using a context $c$ to generate a translation
    
## Speech Recognition
- A microphone records little variations in air pressure over time, and it is these little variations in air pressur that your ear perceives as sould. You can think if an audio recording a long list of numbers measuring the little air pressure changes detected by the microphone. A audio samples at 44100Hz, means that the microophones gives 44100 numbers per secons. 
- Human ear doesn't process raw wave forms, instead human ear process different frequencies. 
- There is a common preprocessing step for an audio- generate a spectrogram which works similarly to human ears. 
- In the spectrum, the horizontal axis is time, the vertical is frequencies, intensity of different colors shows the amount of energy. 
- A spectrum is computed by sliding a sindow over the raw audio signal, and calculate the most active frequencies in each window using a Fourier Transformation.
- In the past days, speech recognition systems were built using phonemes that are a hand engineered basic units of sound. Linguists used to hypothesize that writing down audio in terms of these basic units of sound called phonemes would be the best way to do speech recognition
- In end to end deep learning, phonemes was no longer needed. 
#### Speech Recognization using Attention Model
#### Speech Recognization Using CTC(Connectionist temporal classification) cost model
- In CTC model, the number of iuput are output are the same, but in speech regonization problem, input X tends to be a lot larger than output Y, CTC collape repeated characters not seperated by "_", and "space" stands for the space between words.  